In [1]:
%%bash

pushd ../data

unzip -O CP936 -P D4nd8kHIfK xuelang_round1_test_b_20180802.zip.zip 
unzip -O CP936 -P Y5TzEQLbHD xuelang_round1_test_b.zip 

rm *.zip

popd

~/Desktop/xuelang/data ~/Desktop/xuelang/code
Archive:  xuelang_round1_test_b_20180802.zip.zip
 extracting: xuelang_round1_test_b.zip  
Archive:  xuelang_round1_test_b.zip
   creating: xuelang_round1_test_b/
  inflating: xuelang_round1_test_b/J01_2018.06.13 13_27_46.jpg  
  inflating: xuelang_round1_test_b/J01_2018.06.13 13_32_11.jpg  
 extracting: xuelang_round1_test_b/J01_2018.06.13 13_41_53.jpg  
 extracting: xuelang_round1_test_b/J01_2018.06.13 13_54_01.jpg  
  inflating: xuelang_round1_test_b/J01_2018.06.13 13_57_10.jpg  
 extracting: xuelang_round1_test_b/J01_2018.06.13 13_59_11.jpg  
  inflating: xuelang_round1_test_b/J01_2018.06.13 14_00_46.jpg  
  inflating: xuelang_round1_test_b/J01_2018.06.13 14_19_19.jpg  
  inflating: xuelang_round1_test_b/J01_2018.06.13 14_22_07.jpg  
  inflating: xuelang_round1_test_b/J01_2018.06.13 14_24_21.jpg  
 extracting: xuelang_round1_test_b/J01_2018.06.13 14_27_05.jpg  
 extracting: xuelang_round1_test_b/J01_2018.06.13 14_38_27.jpg  
 extracting:

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

import torch

from dataset import *
from model import *

In [4]:
model = BaselineModel()
model.load_state_dict(torch.load("w.h5"))

In [5]:
root_test = os.path.join("..", "data", "xuelang_round1_test_b")

import torchvision.transforms as transforms
transform_test = transforms.Compose([ 
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225)),
])

loader_test = get_loader(root_test, False, batch_size=32, transform=transform_test)

In [6]:
import torch.nn.functional as F
import pandas as pd


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict():
    result = pd.DataFrame(columns=["filename", "probability"])
    for x, path in loader_test:
        x = x.to(device)
    
        y = model(x)
        y = F.softmax(y, dim=1)
        # here should be 1 - y, but I realized it few days days later after my final submit
        # y = 1 - y.cpu().numpy()[:, 20]
        y = y.cpu().numpy()[:, 20]
        # in my final submit(which gives highest score), I convert prob to 0 and 1
        # but I quickly switch back and resubmit one without this conversion, I don't know which one it was used
        # y = (y > 0.5).astype("float32")
        data = pd.DataFrame({"filename":path, "probability":y})
        result = result.append(data, ignore_index=True)
    return result

with torch.no_grad():
    r = predict()

In [7]:
r.head()

,filename,probability
0,J01_2018.06.13 13_27_46.jpg,9.990281e-01
1,J01_2018.06.13 13_32_11.jpg,7.480204e-10
2,J01_2018.06.13 13_41_53.jpg,9.999659e-01
3,J01_2018.06.13 13_54_01.jpg,9.996051e-01
4,J01_2018.06.13 13_57_10.jpg,1.000000e+00


In [8]:
r["probability"] = r["probability"].round(4).clip(0.0001, 0.9999)

In [9]:
r.head()

,filename,probability
0,J01_2018.06.13 13_27_46.jpg,0.9990
1,J01_2018.06.13 13_32_11.jpg,0.0001
2,J01_2018.06.13 13_41_53.jpg,0.9999
3,J01_2018.06.13 13_54_01.jpg,0.9996
4,J01_2018.06.13 13_57_10.jpg,0.9999


In [10]:
import datetime

filename = "../submit/submit_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + ".csv"
r.to_csv(filename, index=False, float_format='%.4f')